In [ ]:
import locale
import traceback
import pymysql
import dash
from dash import html
from dash.dependencies import Input, Output, State
import rol_usuario_funciones as ruf
import rol_administrador_inserciones_funciones as raif
import rol_administrador_gestion_funciones as ragf
import layout as ly 
import login as log

# Setea la variable LC_ALL al conjunto de código UTF-8 con descripción español España
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

colors = {"graphBackground": "#F5F5F5", "background": "#ffffff", "text": "#000000"}

opciones_tramos = [
    "Mañana", "Tarde", "Noche"
]

opciones_meses = [
    "Enero", "Febrero", "Marzo", "Abril",
    "Mayo", "Junio", "Julio", "Agosto",
    "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

not_uploaded_files = []
uploaded_files = []

df = ruf.crear_dataframe()

Creamos el layout.

In [ ]:
app.layout =  html.Div(
            children=[
               ly.generar_layout_usuario(df),
               ly.generar_layout_login(),
               ly.generar_layout_administrador(),
               ly.generar_layout_elegir_rol()
            ],
        )       

Login, si entra como usuario, cargamos el dataframe y las listas desplegables

In [ ]:
@app.callback(
    Output("layout-login", "style"),
    Output("layout-usuario", "style"),
    Output("layout-administrador", "style"),
    Output("layout-elegir-rol", "style"),
    Output("login-error-message", "children"),
    Input("login-button", "n_clicks"),
    Input("elegir-usuario-button", "n_clicks"),
    Input("elegir-administrador-button", "n_clicks"),
    [
        State("username-input", "value"),
        State("password-input", "value")
    ],
)

def login(n_clicks, n_clicks_rol_usuario, n_clicks_rol_administrador, username, password):
    
    if n_clicks and not n_clicks_rol_usuario and not n_clicks_rol_administrador:
        result = log.comprobar_constraseña(username, password)
        if  result == 1:
            #Elegir rol visible
            return {"display": "none"}, {"display": "none"}, {"display": "none"}, {"display": "block"},""
        elif result == 0:
            #Rol usuario visible
            return {"display": "none"}, {"display": "block"}, {"display": "none"}, {"display": "none"},""
            #Mostramos error en login
        else:
            return {"display": "block"}, {"display": "none"}, {"display": "none"}, {"display": "none"},"Los datos introducidos no son válidos."
        
    elif n_clicks_rol_usuario:
        #Rol usuario visible
        return {"display": "none"}, {"display": "block"}, {"display": "none"}, {"display": "none"},""
    
    elif n_clicks_rol_administrador:
        #Rol administrador visible
        return {"display": "none"}, {"display": "none"}, {"display": "block"}, {"display": "none"},""
    
    else:
        #Nos quedamos en login
        return {"display": "block"}, {"display": "none"}, {"display": "none"}, {"display": "none"},""


Creamos las funciones callback necesarias para rol usuario.

In [ ]:
@app.callback(
    [
        Output("map", "children"),
        Output("output-data-1", "children"),
        Output("output-data-2", "children"),
        Output("output-data-3", "children"),
        Output("output-data-4", "children"),
        Output("output-data-message", "children"),
        Output("output-data-message-mapa", "children")
    ],
    [
        Input("dropdownTramos", "value"),
        Input("dropdownMeses", "value"),
        Input("dropdownAnios", "value"),
        Input("dropdownTipos", "value"),
        Input("dropdownModus", "value"),
        Input("dropdownCalificacion", "value"),
        Input("dropdownDistrito", "value"),
    ],
)

def upload_and_write_data(dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito):
    return ruf.generar_figuras(df, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)

@app.callback(
    Output("informe-output-message", "children"),
    Input("generar-informe-button", "n_clicks"),
    [
        State("dropdownTramos", "value"),
        State("dropdownMeses", "value"),
        State("dropdownAnios", "value"),
        State("dropdownTipos", "value"),
        State("dropdownModus", "value"),
        State("dropdownCalificacion", "value"),
        State("dropdownDistrito", "value"),
    ],
)

def generar_informe(n_clicks, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito):
    if n_clicks:
        return ruf.generar_informe_pdf(df, n_clicks, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)


Creamos las funciones callback necesarias para rol administrador.

In [ ]:
@app.callback(
    Output("output-data-not-upload", "children"),
    Input("upload-data", "filename"),
    Input("upload-data", "contents")
)

def upload_file(filename, contents):
    if filename is not None:
        divs = [
            html.Div([
                f"El archivo {file} ha sido subido.",
            ]) 
        for file in filename]


        return divs
    else:
        return html.Div(["Ningún archivo ha sido subido."])
    
@app.callback(
    Output("output-data-upload", "children"),
    Input("submit-button", "n_clicks"), 
    State("upload-data", "filename"),
    State("upload-data", "contents")
)

def upload_and_write_data(n_clicks, filenames, contents):
    try:
        if(n_clicks > 0):
            listdenuncias = []
            if contents is not None:
                listdenuncias = raif.insert_data(contents)

                # Return a message indicating that the data was written to the database
                for index, file in enumerate(filenames):
                    if listdenuncias[index] == 0:
                        not_uploaded_files.append(file)
                    else:
                        uploaded_files.append(file)
                        
                divs = [html.Div([f"El archivo {file} ha sido resgitrado con éxito en la base de datos."]) for file in uploaded_files]
                divs2 = [html.Div([f"El archivo {file} NO CONTIENE LAS CABECERAS ADECUADAS."]) for file in not_uploaded_files]
                return divs + divs2
    except pymysql.err.InterfaceError as e:
        print(e)
        traceback.print_exc()
        return html.Div(["Ha ocurrido un error al establecer la conexión con la base de datos, disculpe las molestias."])
    except Exception as e:
        print(e)
        traceback.print_exc()
        return html.Div(["Ha ocurrido un error al procesar los datos, disculpe las molestias."])    
            
@app.callback(
    Output("layout-administrador-subida-archivos", "style"),
    Output("layout-administrador-borrado-archivos", "style"),
    Output("layout-administrador-usuarios", "style"),
    Output("dropdownAnios-borrado", "options"),
    Output("table-usuarios", "data"),
    Output("table-usuarios", "selected_rows"),
    Output("output-data-borrado-usuarios", "children"),
    Output("output-data-usuarios", "children"),
    Input("btn-administrador-insercion", "n_clicks"),
    Input("btn-administrador-borrado", "n_clicks"),
    Input("btn-administrador-usuarios", "n_clicks"),
    Input("borrar-usuarios-button", "n_clicks"),
    Input("crear-usuario-button", "n_clicks"),
    [
        State('table-usuarios', 'selected_rows'),
        State('table-usuarios', 'data'),
        State("crear-usuario-username-input", "value"),
        State("crear-usuario-password-input", "value"),
        State("crear-usuario-password-repetida-input", "value"),
        State("dropdownRol", "value")
    ]
)

def controlar_visibilidades(btn_insercion_clicks, btn_borrado_clicks, btn_usuarios_clicks, btn_borrado_usuarios_clicks, btn_crear_usuario_clicks, selected_rows, data_table, username, password, passwordRepetida, administrador):
        # Determine which button was clicked
        ctx = dash.callback_context
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if triggered_id is None or triggered_id == '':
            return {"display": "block"}, {"display": "none"}, {"display": "none"}, [], [], selected_rows, "", ""
        else:
            # Define default styles
            style_insercion = {"display": "none"}
            style_borrado = {"display": "none"}
            style_usuarios = {"display": "none"}
            opciones_años = []
            usuarios = []
            usuarios_texto = ""
            selected_rows_return = selected_rows
            crear_usuarios_texto = ""
            
            # Update styles based on the clicked button
            if triggered_id == "btn-administrador-insercion":
                style_insercion = {"display": "block"}
            elif triggered_id == "btn-administrador-borrado":
                style_borrado = {"display": "block"}
                opciones_años = ragf.obtener_años_borrar()
            elif triggered_id == "btn-administrador-usuarios":
                style_usuarios = {"display": "block"}
                try:
                    usuarios = ragf.obtener_usuarios().to_dict('records')
                except Exception as e:
                    print(e)
                    traceback.print_exc()
                    usuarios_texto =  ["Ha ocurrido un error al obtener los usuarios", html.Br(), "Inténtelo de nuevo"]

            elif triggered_id == "crear-usuario-button":
                style_usuarios = {"display": "block"}
                selected_rows_return = []

                if(username is None or password is None or passwordRepetida is None or administrador is None):
                    crear_usuarios_texto = "Rellene todos los campos"
                elif len(password) < 8:
                    crear_usuarios_texto = "La contraseña debe tener más de 8 caracteres"
                elif password != passwordRepetida:
                    crear_usuarios_texto = "Ambas contraseñas no coinciden"
                elif sum(1 for row in data_table if row['Usuario'] == username) > 0:
                    crear_usuarios_texto = "Ya existe un usuario con ese nombre"
                else:
                    try:
                        ragf.crear_usuario(username, password, administrador)
                        crear_usuarios_texto = f"El usuario '{username}' se ha regitrado con éxito"
                    except Exception as e:
                        print(e)
                        traceback.print_exc()
                        crear_usuarios_texto = ["Ha ocurrido un error al crear el usuario", html.Br(), "Inténtelo de nuevo"]
                    
                try:
                    usuarios = ragf.obtener_usuarios().to_dict('records')
                except Exception as e:
                    print(e)
                    traceback.print_exc()
                    usuarios_texto =  ["Ha ocurrido un error al obtener los usuarios", html.Br(), "Inténtelo de nuevo"]

            elif triggered_id == "borrar-usuarios-button":
                style_usuarios = {"display": "block"}
                try:
                    if selected_rows is None or len(selected_rows) == 0:
                        usuarios_texto = ["Debe seleccionar algún usuario a borrar"]
                    else:
                        administradores = sum(1 for row in data_table if row['Rol'] == 'Administrador')
                        administradores_selecionados = sum(1 for i in selected_rows if data_table[i]['Rol'] == 'Administrador')

                        if administradores - administradores_selecionados > 0:
                            selected_usuarios = [data_table[i]['Usuario'] for i in selected_rows]
                            ragf.borrar_usuarios(selected_usuarios)
                            selected_rows_return = []
                            usuarios_texto = ["Los usuarios han sido borrados con éxito"]
                        else:
                            selected_rows_return = []
                            usuarios_texto = ["No puede borrar todos los administradores."]

                    try:
                        usuarios = ragf.obtener_usuarios().to_dict('records')
                    except Exception as e:
                        print(e)
                        traceback.print_exc()
                        usuarios_texto =  [usuarios_texto, html.Br(), "Ha ocurrido un error al obtener los usuarios", html.Br(), "Inténtelo de nuevo"]

                except Exception as e:
                    print(e)
                    traceback.print_exc()
                    usuarios_texto =  ["Ha ocurrido un error al borrar los usuarios", html.Br(), "Inténtelo de nuevo"]
          
            return style_insercion, style_borrado, style_usuarios,  [{"label": k, "value": k} for k in opciones_años], usuarios, selected_rows_return, usuarios_texto, crear_usuarios_texto
        
#Borrar datos año
@app.callback(
    Output("output-data-borrado", "children"),
    Input("borrar-datos-button", "n_clicks"),
    State("dropdownAnios-borrado", "value")
)

def borrar_datos(btn_borrado_clicks, dropdownAnios_values):
    if (btn_borrado_clicks > 0 and dropdownAnios_values is not None):
        try:
            ragf.borrar_datos(dropdownAnios_values)
            return [f"Los datos han sido borrados con éxito"]
        except Exception as e:
            print(e)
            traceback.print_exc()
            return ["Ha ocurrido un error al borrar los datos", html.Br(), "Inténtelo de nuevo"]



Por último, ejecutamos la aplicación

In [6]:
app.run_server(port="8051")